In [139]:
#Linear regression is the most simple type of nn if you think abnout it
#Basically there is an input layer and an output layer
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

#this one looks at temperature, humidity and rainfall to get crop yields - lets do something like that


inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')


#insulin, blood sugar, blood pressure

metrics = np.array([[100, 130, 220], [50, 60, 150], [30, 130, 221], 
                    [40, 51, 230], [90, 180, 270], [100, 130, 220], 
                    [50, 60, 150],[30, 130, 221], [40, 51, 230], 
                    [90, 180, 270], [100, 130, 220], [50, 60, 150], 
                    [30, 130, 221], [40, 51, 230], [90, 180, 270]], dtype="float32")

relations = np.array([[1,1], [0,1], [0,0],
                      [1,0],[0,1], [0,0],
                      [1,1],[0,0], [0,1],
                      [0,0], [0,0], [0,0],
                      [1,1], [0,0], [0,0]], dtype="float32")

In [140]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
training_dataset = TensorDataset(inputs, targets)
batches = 5
train_dl = DataLoader(training_dataset, batches, shuffle=True)


In [141]:
#inital predictions and loss of the function
model = nn.Linear(3,2)

preds = model(inputs)
print(preds)

mse = F.mse_loss

loss = mse(preds, targets)


tensor([[ -9.8472,  55.5202],
        [ -9.3988,  75.4434],
        [-11.6646,  80.4879],
        [-18.8455,  55.1750],
        [ -1.7883,  74.4142],
        [ -9.8472,  55.5202],
        [ -9.3988,  75.4434],
        [-11.6646,  80.4879],
        [-18.8455,  55.1750],
        [ -1.7883,  74.4142],
        [ -9.8472,  55.5202],
        [ -9.3988,  75.4434],
        [-11.6646,  80.4879],
        [-18.8455,  55.1750],
        [ -1.7883,  74.4142]], grad_fn=<AddmmBackward>)


In [155]:
#need a number of epochs, a model, a loss function, and a training dataloader
def fit(num_epochs, model, loss_fn, train_dl):
    for epoch in range(num_epochs):
        #in 5 different batches, and it has 2 different tensors that are stored
        for x,y in train_dl:
            preds = model(x)
            loss = mse(preds, y)
            loss.backward()
            with torch.no_grad():
                model.weight -= model.weight.grad * 1e-5
                model.bias -= model.bias.grad * 1e-5
                model.weight.grad.zero_()
                model.bias.grad.zero_()

        if (epoch + 1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [156]:
# fit(500, model, mse, train_dl)

fit(100, model, mse, train_dl)

Epoch [10/100], Loss: 0.7378
Epoch [20/100], Loss: 0.5842
Epoch [30/100], Loss: 0.6182
Epoch [40/100], Loss: 0.4965
Epoch [50/100], Loss: 0.4942
Epoch [60/100], Loss: 0.6125
Epoch [70/100], Loss: 0.6593
Epoch [80/100], Loss: 0.5948
Epoch [90/100], Loss: 0.6328
Epoch [100/100], Loss: 0.6870


In [157]:
def make_pred(temp, hum, rain):
    #temperature - humidity - rainfall
    pred = np.array([temp, hum, rain], dtype="float32")
    pred = torch.from_numpy(pred)
    print(pred)
    
    preds = model(pred)
    print(preds)
    

In [158]:
make_pred(73,67,43)

tensor([73., 67., 43.])
tensor([57.1828, 70.4109], grad_fn=<AddBackward0>)
